In [760]:
#import delle librerie
import numpy as np
import nltk
import pandas as pd
from nltk import word_tokenize
from textblob import TextBlob
from nltk.corpus import stopwords
import re
import pylab
from collections import Counter
import itertools
from nltk import bigrams
from nltk.corpus import stopwords
import string
import seaborn as sns
from nltk.stem.lancaster import LancasterStemmer
import matplotlib.pyplot as plt
import matplotlib.style as style
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()
import os
from PIL import Image
import glob
from datetime import datetime
from nltk.sentiment.vader import SentimentIntensityAnalyzer as vad
#nltk.download('punkt')
#nltk.download('stopwords')

In [761]:
df=pd.read_csv('/Users/alessandromotta/Downloads/vax2.csv', encoding='utf8')

/Users/alessandromotta/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning:

Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.



In [762]:
df['location']=df.location.str.lower()
df=df[df.location.str.contains('uk|england|united kingdom' , na=False)]
df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,_id,date,is_RT,location,text,user_id,username,entities
16,16,57,57,57.0,5fcbb347ce2a8e9b3d4d242c,2020-12-05 16:20:18,False,"wales, united kingdom",one trust b0ris know v@xx flawed never voted r...,1.271769e+18,Dee Smith #SackVallance #SackWhitty #TrumpWon,NaN
27,27,105,106,106.0,5fcbb34bce2a8e9b3d4d245d,2020-12-05 16:20:22,False,"lavenham, england","favourite meme day, self diagnosed medical exp...",1.169711e+18,Ryan Lynch,NaN
52,52,173,174,174.0,5fcbb352ce2a8e9b3d4d24a1,2020-12-05 16:20:29,False,uk- wales,six #questions #ask #covid-19 #vaccines | heal...,3.847631e+08,Colin H,NaN
75,75,268,269,269.0,5fcbb35ace2a8e9b3d4d2500,2020-12-05 16:20:37,False,"scarborough, uk",@chillyzag ridiculous get that's think vaccine?,5.925436e+07,LoopyLou,NaN
88,88,308,309,309.0,5fcbb35ece2a8e9b3d4d2528,2020-12-05 16:20:41,False,"s yorkshire, england",@eath1223 take covid vaccine flu vaccine. neit...,2.059544e+07,Yvonne 🐈🇬🇧,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
267621,267621,842350,41532,41532.0,5fe074933f4fea8a8e6413ab,2020-12-21 10:10:22,False,akatsuki,pfizer moderna??,2.411111e+08,Pain Nagato,NaN
267637,267637,842411,41593,41593.0,5fe0749b3f4fea8a8e6413e8,2020-12-21 10:10:30,False,"london, uk",tier 4 covid-19 restriction continue take effe...,3.952675e+07,H&F Council,NaN
267652,267652,842473,41655,41655.0,5fe074a33f4fea8a8e641426,2020-12-21 10:10:38,False,united kingdom,@jeremyvineon5 elderly father law significant ...,1.314626e+18,ClaireStuart,NaN
267653,267653,842474,41656,41656.0,5fe074a33f4fea8a8e641427,2020-12-21 10:10:38,False,uk,@moriartyprofj contradicts hancock &amp; johns...,1.019148e+18,Keith,NaN


In [763]:
del df['Unnamed: 0']
del df['Unnamed: 0.1']
del df['Unnamed: 0.1.1']
del df['Unnamed: 0.1.1.1']
del df['entities']
del df['is_RT']

In [764]:
df = df.rename(columns={'_id':'id'})

In [765]:
df.columns

Index(['id', 'date', 'location', 'text', 'user_id', 'username'], dtype='object')

In [766]:
df.location = df.location.str.lower() 

In [767]:
def clean(x):
    return str(x).encode('ascii', 'ignore').decode('UTF-8')

In [768]:
df.text = df.text.str.lower() 
#convert text to lower-case

df.text=df.text.str.replace(r'((www\.[^\s]+)|(https?://[^\s]+))', r'', regex=True)
#toglie link

df.text=df.text.str.replace(r'@[^\s]+', r'', regex=True)
#toglie username

df.text=df.text.str.replace(r'\n', r' ', regex=True)
#sostituisce \n con spazio

df.text=df.text.str.replace(r'#([^\s]+)', r'\1', regex=True)
#toglie hashtags

#df['tokenized_text'] =  df['text'].apply(word_tokenize) 
#tokenize in una nuova colonna

In [769]:
df.text=df.text.str.replace(r'(<br\s*/><br\s*/>)|(\-)|(\/)', r' ', regex=True)

#elimina simboli

In [770]:
stop_words=[]

In [771]:
emoticons=[]

In [772]:
df['text'] = df['text'].astype(str)

In [773]:
def clean_tweets(tweet):
 
    stop_words = set(stopwords.words('english'))
    word_tokens = nltk.word_tokenize(str(df.text))
#after tweepy preprocessing the colon symbol left remain after removing mentions
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
#replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
#filter using NLTK library append it to a string
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []
#looping through conditions
    for w in word_tokens:
#check tokens against stop words
        if w not in stop_words and w not in emoticons:
            filtered_tweet.append(w)
    return ' '.join(filtered_tweet)
    #print(word_tokens)
    #print(filtered_sentence)return tweet

In [774]:
df.text.apply(lambda x : clean_tweets(x))

16        16 one trust b0ris know v flawed never voted r...
27        16 one trust b0ris know v flawed never voted r...
52        16 one trust b0ris know v flawed never voted r...
75        16 one trust b0ris know v flawed never voted r...
88        16 one trust b0ris know v flawed never voted r...
                                ...                        
267621    16 one trust b0ris know v flawed never voted r...
267637    16 one trust b0ris know v flawed never voted r...
267652    16 one trust b0ris know v flawed never voted r...
267653    16 one trust b0ris know v flawed never voted r...
267664    16 one trust b0ris know v flawed never voted r...
Name: text, Length: 21825, dtype: object

In [775]:

stop =stopwords.words('english')
stop[:20]

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his']

In [776]:
punctuation = string.punctuation
print(punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


# Stemming

In [777]:
# Function to get the counter
def get_counter(df):
  sentences = (list(itertools.chain(df)))
  flat_list = [item for sublist in sentences for item in sublist]
  c = Counter(flat_list)
  return c

In [778]:


lancaster_stemmer = LancasterStemmer()


def preprocess(tokenized_sentence, stopwords=[]):
  tokenized_sentence_stop = [item.lower() for item in tokenized_sentence if item.lower() not in set(stopwords)] #remove stopwords and lowercase
  x = [item.lower() in for item in tokenized_sentence_stop if item not in punctuation] #remove punctuation 
  return [lancaster_stemmer.stem(item) for item in x]


tweets_tokenized_prep = df.text.apply(lambda x: preprocess(x, stopwords=stop))

SyntaxError: invalid syntax (<ipython-input-778-6c664ade8831>, line 6)

In [ ]:
c = get_counter(tweets_tokenized_prep)
c.most_common(30)

In [ ]:
sentiment=vad()

sen=['Positive','Negative','Neutral']
sentiments=[sentiment.polarity_scores(i) for i in df['text'].values]
df['Negative Score']=[i['neg'] for i in sentiments]
df['Positive Score']=[i['pos'] for i in sentiments]
df['Neutral Score']=[i['neu'] for i in sentiments]
df['polarity']=[i['compound'] for i in sentiments]
score=df['polarity'].values
t=[]
for i in score:
    if i >=0.05 :
        t.append('Positive')
    elif i<=-0.05 :
        t.append('Negative')
    else:
        t.append('Neutral')
df['Overall Sentiment']=t

# Moderna vs Pfizer

In [ ]:
df.dropna(axis=1, inplace=True)

#vengono definiti i tweets negativi e quelli positivi
df['posneg']=df['polarity'].apply(lambda x: 'pos' if x>=0 else 'neg')

df['date']= pd.DatetimeIndex(df['date']) + pd.offsets.Hour(2)
df.set_index(pd.DatetimeIndex(df['date']), inplace=True)

#Vengono selezionati solo i tweets che parlano di Moderna e di Pfizer
data_moderna = df[df['text'].str.contains('moderna', case=False)]
data_moderna = data_moderna[data_moderna['text'].str.contains('pfizer', case=False) == False]

data_pf = df[df['text'].str.contains('pfizer', case=False)]
data_pf = data_pf[data_pf['text'].str.contains('moderna', case=False) == False]

In [ ]:
#Viene fatta la media per data per poter visualizzare i dati
polarity_series_moderna = data_moderna['polarity']
polarity_series_moderna = polarity_series_moderna.resample('D').mean()

polarity_series_pf = data_pf['polarity']
polarity_series_pf = polarity_series_pf.resample('D').mean()

polarity_series_moderna.dropna(inplace=True)
polarity_series_pf.dropna(inplace=True)

In [ ]:


moderna_sentita = data_moderna['polarity'].values
pfizer_sentita = data_pf['polarity'].values

fig = plt.figure()
ax = fig.add_subplot(111)

ax.hist([moderna, pfizer], color=['red', 'blue'], label=['Moderna', 'pfizer'])
ax.legend(loc='upper left')
ax.set_xlabel('Sentiment')
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
import matplotlib.pyplot as plt


fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(111)
ax.set_title('Moderna vs Pfizer Sentiment')


polarity_series_moderna.plot(ax=ax, color='red', label='Moderna_sentiment')
polarity_series_pf.plot(ax=ax, color='Blue', label='Pfizer_sentiment')
ax.legend(loc='lower right')
ax.set_xlabel('Time')
ax.set_ylabel('Sentiment')
#plt.savefig('ModernaVSPfizer.png')

In [ ]:
df.polarity.value_counts()

In [ ]:
mod = data_moderna['polarity']
pfi = data_pf['polarity']

print('Moderna sentiment mean: ' + str(round(mod.mean(), 3)))
print('Pfizer sentiment mean:' + str(round(pfi.mean(), 3)))
print('\n')

print('Moderna sentiment standard deviation: ' + str(round(mod.std(), 3)))
print('Pfizer sentiment standard deviation: ' + str(round(pfi.std(), 3)))

In [ ]:
Mod_pos = mod[mod > 0].size
Mod_neg = mod[mod < 0].size
pfi_pos = pfi[pfi > 0].size
pfi_neg = pfi[pfi < 0].size

print('Moderna:\n')
print('Positive tweets: ' + str(Mod_pos))
print('Negative tweets: ' + str(Mod_neg))
print('Percentage of tweets positive: ' + str(round((Mod_pos/(Mod_neg+Mod_pos))*100, 1)) + '%\n')

print('Pfizer:\n')
print('Positive tweets: ' + str(pfi_pos))
print('Negative tweets: ' + str(pfi_neg))
print('Percentage of tweets positive: ' + str(round((pfi_pos/(pfi_neg+pfi_pos))*100, 1)) + '%\n')

print('TOTAL:\n')
print('Positive tweets: ' + str(pfi_pos + Mod_pos))
print('Negative tweets: ' + str(pfi_neg + Mod_neg))
print('Percentage of tweets positive: ' + str(round((Mod_pos+pfi_pos)/(Mod_pos+pfi_pos+Mod_neg+pfi_neg)*100, 1)) + '%')

# JHONSON VS HANCOCK

In [ ]:
df.dropna(axis=1, inplace=True)

#vengono definiti i tweets negativi e quelli positivi
df['posneg']=df['polarity'].apply(lambda x: 'pos' if x>=0 else 'neg')

df['date']= pd.DatetimeIndex(df['date']) + pd.offsets.Hour(2)
df.set_index(pd.DatetimeIndex(df['date']), inplace=True)

#Vengono selezionati solo i tweets che parlano di Moderna e di Pfizer
data_boris = df[df['text'].str.contains('boris', case=False)]
data_boris = data_boris[data_boris['text'].str.contains('hancock|matt', case=False) == False]

data_han = df[df['text'].str.contains('hancock|matt', case=False)]
data_han = data_han[data_han['text'].str.contains('boris', case=False) == False]

#Viene fatta la media per data per poter visualizzare i dati
polarity_series_boris = data_boris['polarity']
polarity_series_boris = polarity_series_boris.resample('D').mean()

polarity_series_han = data_han['polarity']
polarity_series_han = polarity_series_han.resample('D').mean()

polarity_series_boris.dropna(inplace=True)
polarity_series_han.dropna(inplace=True)

In [ ]:
import matplotlib.pyplot as plt

boris = data_boris['polarity'].values
hancock = data_han['polarity'].values

fig = plt.figure()
ax = fig.add_subplot(111)

ax.hist([boris, hancock], color=['red', 'blue'], label=['Boris Johnson', 'Matt Hancock'])
ax.legend(loc='upper left')
ax.set_xlabel('Sentiment')
ax.set_ylabel('Frequency')
plt.show()



In [ ]:
import matplotlib.pyplot as plt


fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(111)
ax.set_title('Boris Johnson vs Matt Hancock Sentiment')


polarity_series_boris.plot(ax=ax, color='red', label='Boris Johnson')
polarity_series_han.plot(ax=ax, color='Blue', label='Matt Hancock')
ax.legend(loc='lower right')
ax.set_xlabel('Time')
ax.set_ylabel('Sentiment')
#plt.savefig('johnVSHanck.png')

# Sky vs BBC

In [ ]:
df.dropna(axis=1, inplace=True)

#vengono definiti i tweets negativi e quelli positivi
df['posneg']=df['polarity'].apply(lambda x: 'pos' if x>=0 else 'neg')

df['date']= pd.DatetimeIndex(df['date']) + pd.offsets.Hour(2)
df.set_index(pd.DatetimeIndex(df['date']), inplace=True)

#Vengono selezionati solo i tweets che parlano di Moderna e di Pfizer
data_bbc = df[df['text'].str.contains('bbcnews|bbc', case=False)]
data_bbc = data_bbc[data_bbc['text'].str.contains('skynews|sky', case=False) == False]

data_sky = df[df['text'].str.contains('skynews|sky', case=False)]
data_sky = data_sky[data_sky['text'].str.contains('bbcnews|bbc', case=False) == False]

data_cnn = df[df['text'].str.contains('cnn', case=False)]
#data_cnn = data_cnn[data_cnn['text'].str.contains('skynews|sky|bbc', case=False) == False]

#Viene fatta la media per data per poter visualizzare i dati
polarity_series_bbc = data_bbc['polarity']
polarity_series_bbc = polarity_series_bbc.resample('D').mean()

polarity_series_sky = data_sky['polarity']
polarity_series_sky = polarity_series_sky.resample('D').mean()

polarity_series_cnn = data_cnn['polarity']
polarity_series_cnn = polarity_series_cnn.resample('D').mean()

polarity_series_bbc.dropna(inplace=True)
polarity_series_cnn.dropna(inplace=True)
polarity_series_sky.dropna(inplace=True)

In [ ]:
data_cnn = data_cnn.loc[data_cnn['id'] != 0]

In [ ]:
import matplotlib.pyplot as plt

bbcnews = data_bbc['polarity'].values
skynews = data_sky['polarity'].values

fig = plt.figure()
ax = fig.add_subplot(111)

ax.hist([bbcnews, skynews], color=['red', 'blue'], label=['BBCnews', 'Skynews'])
ax.legend(loc='upper left')
ax.set_xlabel('Sentiment')
ax.set_ylabel('Frequency')
plt.show()




In [ ]:
import matplotlib.pyplot as plt


fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(111)
ax.set_title('BBCnews vs Skynews vs CNN')


polarity_series_bbc.plot(ax=ax, color='red', label='BBCnews')
polarity_series_sky.plot(ax=ax, color='Blue', label='Skynews')
polarity_series_cnn.plot(ax=ax, color='Green', label='CNN')
ax.legend(loc='lower right')
ax.set_xlabel('Time')
ax.set_ylabel('Sentiment')